# Deploying an already trained model from PMML into IBM Watson Machine Learning
## in a notebook in Watson Studio Local

See also: <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener noreferrer">Watson Machine Learning Python client</a>


### Step 1: Authenticate with the Watson Machine Learning Python client

In the following cell, paste three pieces of information:

<ul>
<li><b>url</b> : The base URL of your Watson Studio Local server
<p>For example, if the URL of your Watson Studio Local server is:<br/><pre>https://111.22.333.444:5555/home?context=wdp</pre></p>
<p>Then the "url" in the following cell should be:<br/><pre>https://111.22.333.444:5555</pre></p></li>
</ul>
<ul>
<li><p><b>username</b> : The username that you use to sign in to Watson Studio Local</p></li>
</ul>
<ul>
<li><p><b>password</b> : The password that you use to sign in to Watson Studio Local</p></li>
</ul>

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = {
    "instance_id" : "icp",
    "url"         : "https://9.30.117.150",
    "username"    : "",
    "password"    : ""
}
client = WatsonMachineLearningAPIClient( wml_credentials )

### Step 2: Download a sample PMML file to the notebook working directory

**About the sample PMML file and model**

The sample PMML file was generated by this notebook: <a href="https://github.com/pmservice/wml-sample-models/blob/master/spark/import-pmml/spark-mllib-export-pmml-scala.ipynb" target="_blank" rel="noopener noreferrer">Saving a Spark MLlib model in PMML format</a>

The sample model is a logistic regression model for predicting whether or not a customer will purchase a tent from a fictional outdoor equipment store, based on the customer charateristics.

The data used to train the model is the "GoSales.csv" training data in the IBM Watson Studio community: <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/aa07a773f71cf1172a349f33e2028e4e" target="_blank" rel="noopener noreferrer">GoSales sample data</a>

In [4]:
import wget
pmml_file_url = 'https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/import-pmml/spark-mllib-lr-model-pmml.xml'
pmml_filename = wget.download( pmml_file_url )
print( pmml_filename )

In [7]:
!cat spark-mllib-lr-model-pmml.xml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_2" version="4.2">
    <Header description="logistic regression">
        <Application name="Apache Spark MLlib" version="2.3.0"/>
        <Timestamp>2018-12-24T00:33:53</Timestamp>
    </Header>
    <DataDictionary numberOfFields="5">
        <DataField name="field_0" optype="continuous" dataType="double"/>
        <DataField name="field_1" optype="continuous" dataType="double"/>
        <DataField name="field_2" optype="continuous" dataType="double"/>
        <DataField name="field_3" optype="continuous" dataType="double"/>
        <DataField name="target" optype="categorical" dataType="string"/>
    </DataDictionary>
    <RegressionModel modelName="logistic regression" functionName="classification" normalizationMethod="logit">
        <MiningSchema>
            <MiningField name="field_0" usageType="active"/>
            <MiningField name="field_1" usageType="active"/>
            <MiningFi

### Step 3: Store the model

In [9]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Model from PMML",
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.RUNTIME_UID: "pmml_4.2.1"
}

In [10]:
model_details = client.repository.store_model( model=pmml_filename, meta_props=metadata )

### Step 4: Deploy the model

In [11]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Model deployment from PMML",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [13]:
model_id = model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_id, meta_props=metadata )



#######################################################################################

Synchronous deployment creation for uid: 'c9630a7a-9920-460d-a4e6-b972438a6c89' started

#######################################################################################


initializing


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5a2c6a00-c178-4180-bd15-3432cd3bc222'
------------------------------------------------------------------------------------------------




### Step 5: Send sample payload data to make a prediction

In [14]:
deployment_id = deployment_details["metadata"]["guid"]

In the <a href="https://github.com/pmservice/wml-sample-models/blob/master/spark/import-pmml/spark-mllib-export-pmml-scala.ipynb" target="_blank" rel="noopener noreferrer">sample notebook that builds the model and saves it in PMML format</a>, you can see examples of sample data:

Features of a customer who did not buy a tent:
```
| GENDER | AGE | MARITAL_STATUS | PROFESSION     | feature_vector           |
+--------+-----+----------------+----------------+--------------------------+
| "F"    | 35  | "Married"      | "Professional" | [ 1.0, 35.0 , 0.0, 1.0 ] |
```

Features of a customer who did buy a tent:
```
| GENDER | AGE | MARITAL_STATUS | PROFESSION     | feature_vector           |
+--------+-----+----------------+----------------+--------------------------+
| "M"    | 20  | "Single"       | "Sales"        | [ 0.0, 20.0, 1.0, 2.0 ]  |
```

It is the four numeric values in `feature_vector` column that are sent to the model to make a prediction.

The values in the `feature_vector` column for the "GENDER", "MARITAL_STATUS", and "PROFESSION" columns are generated by string indexers:
```
+------+------------+      +--------------+--------------------+      +------------+----------------+
|GENDER|GENDER_index|      |MARITAL_STATUS|MARITAL_STATUS_index|      |  PROFESSION|PROFESSION_index|
+------+------------+      +--------------+--------------------+      +------------+----------------+
|     M|         0.0|      |       Married|                 0.0|      |       Other|             0.0|
|     F|         1.0|      |        Single|                 1.0|      |Professional|             1.0|
+------+------------+      |   Unspecified|                 2.0|      |       Sales|             2.0|
                           +--------------+--------------------+      |   Executive|             3.0|
                                                                      |      Trades|             4.0|
                                                                      | Hospitality|             5.0|
                                                                      |     Student|             6.0|
                                                                      |      Retail|             7.0|
                                                                      |     Retired|             8.0|
                                                                      +------------+----------------+
```

In [19]:
negative_example_payload = { "scoring_input_data" : [ { "fields" : [ "field_0", "field_1", "field_2", "field_3" ], "values" : [ [ 1.0, 35.0, 0.0, 1.0 ] ] } ] }
positive_example_payload = { "scoring_input_data" : [ { "fields" : [ "field_0", "field_1", "field_2", "field_3" ], "values" : [ [ 0.0, 20.0, 1.0, 2.0 ] ] } ] }

In [20]:
client.deployments.score( deployment_id, negative_example_payload )

{'predictions': [{'fields': ['$RegressionModel-target',
    '$RegressionModelC-target',
    '$RegressionModelP-1',
    '$RegressionModelP-0'],
   'values': [['0',
     0.9367945715178595,
     0.06320542848214054,
     0.9367945715178595]]}]}

In [21]:
client.deployments.score( deployment_id, positive_example_payload )

{'predictions': [{'fields': ['$RegressionModel-target',
    '$RegressionModelC-target',
    '$RegressionModelP-1',
    '$RegressionModelP-0'],
   'values': [['1',
     0.7350517656297156,
     0.7350517656297156,
     0.2649482343702845]]}]}

## Summary
In this notebook, you imported a model into Watson Machine Learning from a sample PMML file using the Watson Machine Learning Python client.

Authors

**Sarah Packowski** is a member of the IBM Watson Studio Content Design team in Canada.

<hr>

Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.